In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt


cleaned_data = pd.read_csv('Cleaned_Plant_2_Data.csv')


target = 'AC_POWER'
X = cleaned_data.drop(columns=[target])
y = cleaned_data[target]


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


def gradient_descent(X, y, learning_rate, num_iterations):
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0
    cost_history = []

    for i in range(num_iterations):
        y_pred = np.dot(X, weights) + bias
        
        cost = (1 / (2 * m)) * np.sum((y_pred - y) ** 2)
        cost_history.append(cost)

        dW = (1 / m) * np.dot(X.T, (y_pred - y))
        db = (1 / m) * np.sum(y_pred - y)

        weights -= learning_rate * dW
        bias -= learning_rate * db

        if i % 100 == 0:
            print(f"Iteration {i}: Cost {cost:.4f}")

    return weights, bias, cost_history


In [ ]:
learning_rates = [0.001, 0.01, 0.1]
num_iterations = 1000
best_cost = float('inf')
best_weights = None
best_bias = None
best_lr = None

for lr in learning_rates:
    print(f"\nTraining with Learning Rate: {lr}")
    weights, bias, cost_history = gradient_descent(X_train_scaled, y_train.values, lr, num_iterations)
    
    final_cost = cost_history[-1]
    print(f"Final Cost for Learning Rate {lr}: {final_cost:.4f}")
    
    if final_cost < best_cost:
        best_cost = final_cost
        best_weights = weights
        best_bias = bias
        best_lr = lr

    plt.plot(cost_history, label=f'LR={lr}')
    plt.xlabel('Iterations')
    plt.ylabel('Cost')
    plt.title('Cost Reduction Over Iterations')
    plt.legend()

plt.show()

print("\nBest Learning Rate:", best_lr)
print("Best Final Cost:", best_cost)
print("Best Bias:", best_bias)

print("\nBest Weights for each feature:")
for feature, weight in zip(X.columns, best_weights):
    print(f"  {feature}: {weight}")

In [ ]:
def predict(X, weights, bias):
    return np.dot(X, weights) + bias

y_pred = predict(X_test_scaled, best_weights, best_bias)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Actual AC Power")
plt.ylabel("Predicted AC Power")
plt.title("Actual vs Predicted AC Power")
plt.show()


In [ ]:
y_val_pred = np.dot(X_val_scaled, weights) + bias
val_mae = mean_absolute_error(y_val, y_val_pred)
val_mse = mean_squared_error(y_val, y_val_pred)
val_rmse = np.sqrt(val_mse)
val_r2 = r2_score(y_val, y_val_pred)

print("\nValidation Set Results:")
print(f"  MAE: {val_mae:.4f}")
print(f"  MSE: {val_mse:.4f}")
print(f"  RMSE: {val_rmse:.4f}")
print(f"  R²: {val_r2:.4f}")

y_test_pred = np.dot(X_test_scaled, weights) + bias
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print("\nTest Set Results:")
print(f"  MAE: {test_mae:.4f}")
print(f"  MSE: {test_mse:.4f}")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  R²: {test_r2:.4f}")

In [ ]:
sample_input = pd.DataFrame({
    'AMBIENT_TEMPERATURE': [29.572043620689648],   
    'MODULE_TEMPERATURE': [41.47150403448277],   
    'IRRADIATION': [0.6542055951724136],         
    'hour': [9],                
    'day': [15],                   
    'month': [5],                             
    'temp_irradiation': [19.34619639733785],          
    'module_temp_irradiation': [27.130889979573954]    
})


sample_input_scaled = scaler.transform(sample_input)
predicted_ac_power = predict(sample_input_scaled, best_weights, best_bias)
print("Predicted AC Power for the given input features:", predicted_ac_power[0])
